In [46]:
import numpy as np 
import pandas as pd


import subprocess
from subprocess import check_output
import gc

In [3]:
files=check_output(["ls", "../pdata"]).decode("utf8")

In [4]:
files

'holidays_events.csv\nitems.csv\noil.csv\nstores.csv\ntrain.csv\ntransactions.csv\n'

#### Let's check the number of rows in each file

In [5]:
for file in files.split('\n'):
    if(file!=''):
        path = '../pdata/'+file
        popen = subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        result,error= popen.communicate()
        print("The file :",file,"has :",result.strip().split()[0],"rows")

The file : holidays_events.csv has : b'351' rows
The file : items.csv has : b'4101' rows
The file : oil.csv has : b'1219' rows
The file : stores.csv has : b'55' rows
The file : train.csv has : b'125497041' rows
The file : transactions.csv has : b'83489' rows


### Memory optimization
##### Since train.csv has 125 mil records, it is best to consider performing some data engineering before starting any analysis.

Following are the steps that have been used to reduce memory consumption


- Check the range of values present for the variables
- The link https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html is referred to find the suitable data type for the values
- Change datatype accordingly
- Split date column into three columns (Day, Month, Year)
    - Reasons to do this
        - Any pandas libriary api's involving operations on column of type datetime is not vectorized. Hence, any such operations take more time.
        - And, Splitting it into three columns will provide better memory utilization.
        
- Impute the on promotion column
- join all the resulting columns

In [3]:
gc.collect()

0

In [2]:
import time
start=time.time()

df = pd.read_csv("../pdata/train.csv")

end = time.time()

print('The import with default options took {} seconds'.format(end-start))
print('The memory consumed by the loaded data is {} gigabytes'.format(df.memory_usage(index=True).sum() / 1024**3))

/Users/nikhilcr/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The import with default options took 148.17780113220215 seconds
The memory consumed by the loaded data is 5.610154941678047 gigabytes


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
id             int64
date           object
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    object
dtypes: float64(1), int64(3), object(2)
memory usage: 5.6+ GB


In [71]:
print("The memory used by numerical columns with pandas default data type presets \n")
total_mem = 0
for col in df.columns:
    if df[col].dtype.name != 'object':
        col_mem = df[col].memory_usage()/1024**2
        total_mem += col_mem
        
        print('Column "{}" is imported as {} and uses {:03.2f} MB'
              .format(col,df[col].dtype.name,col_mem))

print('\nThe total memory consumed by the numerical columns is {:03.2f} GB'.format(total_mem/1024))

The memory used by numerical columns with pandas default data type presets 

Column "id" is imported as int64 and uses 957.47 MB
Column "store_nbr" is imported as int64 and uses 957.47 MB
Column "item_nbr" is imported as int64 and uses 957.47 MB
Column "unit_sales" is imported as float64 and uses 957.47 MB

The total memory consumed by the numerical columns is 3.74 GB


In [3]:
print("The memory used by object columns with pandas default data type presets \n")
total_mem_obj = 0
for col in df.columns:
    if df[col].dtype.name == 'object':
        col_mem = df[col].memory_usage()/1024**2
        total_mem_obj += col_mem
        
        print('Column "{}" is imported as {} and uses {:03.2f} MB'
              .format(col,df[col].dtype.name,col_mem))

print('\nThe total memory consumed by the object columns is {:03.2f} GB'.format(total_mem_obj/1024))

The memory used by object columns with pandas default data type presets 

Column "date" is imported as object and uses 957.47 MB
Column "onpromotion" is imported as object and uses 957.47 MB

The total memory consumed by the object columns is 1.87 GB


In [118]:
print("The memory used by object columns with pandas default data type presets \n")
total_mem_obj = 0
for col in df.columns:
    if df[col].dtype.name != 'object':
        col_mem = df[col].memory_usage()/1024**2
        total_mem_obj += col_mem
        
        print('Column "{}" is imported as {} and uses {:03.2f} MB'
              .format(col,df[col].dtype.name,col_mem))

print('\nThe total memory consumed by the object columns is {:03.2f} GB'.format(total_mem_obj/1024))

The memory used by object columns with pandas default data type presets 

Column "id" is imported as int64 and uses 957.47 MB
Column "date" is imported as datetime64[ns] and uses 957.47 MB
Column "store_nbr" is imported as int64 and uses 957.47 MB
Column "item_nbr" is imported as int64 and uses 957.47 MB
Column "unit_sales" is imported as float64 and uses 957.47 MB

The total memory consumed by the object columns is 4.68 GB


In [7]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [18]:
df.describe()

,id,store_nbr,item_nbr,unit_sales
count,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08
mean,6.274852e+07,2.746458e+01,9.727692e+05,8.554865e+00
std,3.622788e+07,1.633051e+01,5.205336e+05,2.360515e+01
min,0.000000e+00,1.000000e+00,9.699500e+04,-1.537200e+04
25%,3.137426e+07,1.200000e+01,5.223830e+05,2.000000e+00
50%,6.274852e+07,2.800000e+01,9.595000e+05,4.000000e+00
75%,9.412278e+07,4.300000e+01,1.354380e+06,9.000000e+00
max,1.254970e+08,5.400000e+01,2.127114e+06,8.944000e+04


In [29]:
# delete the 5.6+ GB dataframe and garbage collect the stale objects
#del(df)
gc.collect()

# dtype dictionary that can be used for loading based on data above
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }

df1 = pd.read_csv("../pdata/train.csv",dtype=dtype_dict,usecols=[0,2,3,4])


In [77]:
print("The memory used by numerical columns with pandas after optimization \n")
total_mem_post = 0
for col in df1.columns:
    col_mem = df1[col].memory_usage()/1024**2
    total_mem_post += col_mem
    print('Column "{}" is imported as {} and uses {:03.2f} MB'
              .format(col,df[col].dtype.name,col_mem))
print('\nThe total memory consumed by the numerical columns after optimization is {:03.2f} GB'
      .format(total_mem_post/1024))

The memory used by numerical columns with pandas after optimization 

Column "id" is imported as int64 and uses 478.73 MB
Column "store_nbr" is imported as int64 and uses 119.68 MB
Column "item_nbr" is imported as int64 and uses 478.73 MB
Column "unit_sales" is imported as float64 and uses 478.73 MB

The total memory consumed by the numerical columns after optimization is 1.52 GB


In [108]:
print('Improvement after optimization\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ')
for col in df1.columns:
    imp = (df[col].memory_usage() - df1[col].memory_usage()) / df[col].memory_usage()
    imp = imp * 100
    print('Column "{}" has seen {:03.2f} % improvement after optimization'
              .format(col,imp))

print('- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - \
\nTotal memory saved is {:03.2f} GB \nTotal Improvement is {:03.2f} %'
      .format((total_mem-total_mem_post)/1024,(total_mem-total_mem_post)*100/total_mem))

Improvement after optimization
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Column "id" has seen 50.00 % improvement after optimization
Column "store_nbr" has seen 87.50 % improvement after optimization
Column "item_nbr" has seen 50.00 % improvement after optimization
Column "unit_sales" has seen 50.00 % improvement after optimization
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Total memory saved is 2.22 GB 
Total Improvement is 59.37 %


In [109]:
gc.collect()

859

In [110]:
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()

,date,onpromotion
count,125497040,103839389
unique,1684,2
top,2017-07-01,False
freq,118194,96028767


In [4]:
df2 = pd.read_csv("../pdata/train.csv",usecols=[1,5],parse_dates=[0])

# Processing Date Column
df2['Year'] = pd.DatetimeIndex(df2['date']).year
df2['Month'] = pd.DatetimeIndex(df2['date']).month
df2['Day'] =pd.DatetimeIndex(df2['date']).day

# Let's get rid of date column since we have different columns for Day, Month and Year
del(df2['date'])

#data types of each of the sub columns of date
df2['Day']=df2['Day'].astype(np.uint8)
df2['Month']=df2['Month'].astype(np.uint8)
df2['Year']=df2['Year'].astype(np.uint16)


#fill missing values of on promotion by -1
df2["onpromotion"].fillna(-1, inplace=True)
df2["onpromotion"]=df2["onpromotion"].astype(np.int8)

/Users/nikhilcr/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df2['onpromotion'].unique()

array([-1,  0,  1])

In [13]:
df.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion'], dtype='object')

In [27]:
total_mem_before_optimization = 0
total_mem_after_optimization = 0
for col in df.columns:
    if(df[col].dtype.name == 'object'):
        total_mem_before_optimization += df[col].memory_usage()/1024**3
        
for col in df2.columns:
    total_mem_after_optimization+=df2[col].memory_usage() / 1024**3

print('Total memory consumed by date and onpromotion columns before optimization : {:03.2f} GB'
      .format(total_mem_before_optimization))
print('Total memory consumed by date and onpromotion columns after optimization : {:03.2f} GB'
      .format(total_mem_after_optimization))

print('Memory utilization in percentage : {:03.2f} %'
      .format((total_mem_before_optimization-total_mem_after_optimization)*100/total_mem_before_optimization))

Total memory consumed by date and onpromotion columns before optimization : 1.87 GB
Total memory consumed by date and onpromotion columns after optimization : 0.58 GB
Memory utilization in percentage : 68.75 %


In [30]:
# joining part one and two
df = pd.concat([df1.reset_index(drop=True), df2], axis=1)

#drop temp data frames
del(df1)
del(df2)

#Id is just an indicator column, hence not required for our analysis and a not required information
id=df['id']
del(df['id'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 7 columns):
store_nbr      uint8
item_nbr       uint32
unit_sales     float32
onpromotion    int8
Year           uint16
Month          uint8
Day            uint8
dtypes: float32(1), int8(1), uint16(1), uint32(1), uint8(3)
memory usage: 1.6 GB


In [45]:
#memory used of Dataframe-df after processng
post_process_size =df.memory_usage().sum() / 1024**3

print("Memory used by data frame before processing was about 5.6 GB")
print("New data frame uses only {:3.2f} GB".format(post_process_size))
print("Approximate Memory saved is {:3.2f}".format(5.6-post_process_size)," GB")
print("Memory Utilization is {:3.2f} %".format((5.6-post_process_size)*100/5.6))

Memory used by data frame before processing was about 5.6 GB
New data frame uses only 1.64 GB
Approximate Memory saved is 3.96  GB
Memory Utilization is 70.78 %
